## Testing and Debugging

In [1]:
import nn.classifier
import nn.unet_origin as unet_origin
import nn.unet as unet
import torch.optim as optim
import helpers

import torch
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler, SequentialSampler

import img.augmentation as aug
from data.fetcher import DatasetFetcher
import nn.classifier
from nn.train_callbacks import TensorboardVisualizerCallback, TensorboardLoggerCallback, ModelSaverCallback
from nn.test_callbacks import PredictionsSaverCallback

import os
from multiprocessing import cpu_count

from data.dataset import TrainImageDataset, TestImageDataset
import multiprocessing


#### Parameters

In [2]:
# Hyperparameters
input_img_resize = (64, 64) # The resize size of the input images of the neural net
output_img_resize = (64, 64) # The resize size of the output images of the neural net
batch_size = 2
epochs = 150
threshold = 0.5
validation_size = 0.1
sample_size = None

# -- Optional parameters
threads = cpu_count()
use_cuda = torch.cuda.is_available()
# script_dir = os.path.dirname(os.path.abspath(__file__))
script_dir = 'C:\\User\\ZhangX1\\Documents\\Python_Scripts\\DeepLearn\\src'


#### Test DataLoader

In [3]:
# Fetch the datasets
ds_fetcher = DatasetFetcher()
ds_fetcher.fetch_dataset()

# Get the path to the files for the neural net
# We don't want to split train/valid for KFold crossval
X_train, y_train, X_valid, y_valid = ds_fetcher.get_train_files(sample_size=sample_size,
                                                                validation_size=validation_size)
full_x_test = ds_fetcher.get_test_files(sample_size)

# -- Computed parameters
# Get the original images size (assuming they are all the same size)
origin_img_size = ds_fetcher.get_image_size(X_train[0])
    
train_ds = TrainImageDataset(X_train, y_train, input_img_resize, X_transform=None)
train_loader = DataLoader(train_ds, batch_size,
                          sampler=RandomSampler(train_ds),
                          num_workers=threads,
                          pin_memory=use_cuda)


All datasets are present.


#### Opening training images

In [4]:
import numpy as np
import torch.utils.data as data
from PIL import Image

import img.transformer as transformer

img = Image.open(train_loader.dataset.X_train[0])

In [5]:
img_resize = (64, 64)
img = transformer.center_cropping_resize(img, img_resize)
img = np.asarray(img.convert("L"), dtype=np.float32)

In [6]:
# img.shape
img

array([[ 30.,  26.,  23., ...,  10.,  93., 188.],
       [ 34.,  28.,  22., ...,  67., 102., 100.],
       [ 33.,  26.,  22., ...,  90., 116., 105.],
       ...,
       [ 34.,  52.,  75., ...,  90.,  97., 118.],
       [ 35.,  47.,  64., ...,  92., 106.,  66.],
       [ 38.,  48.,  67., ...,  95., 122.,  13.]], dtype=float32)

#### Opening masks

In [59]:
mask = Image.open(train_loader.dataset.y_train_masks[0])
mask = transformer.center_cropping_resize(mask, img_resize)
mask = np.asarray(mask.convert("L"), dtype=np.float32)  # GreyScale

In [60]:
mask

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)

In [61]:
np.unique(mask)

array([1., 2.], dtype=float32)

#### Convert training image and mask to tensor

In [10]:
threshold = 0.5
tensor_img = transformer.image_to_tensor(img)
#tensor_mask = transformer.mask_to_tensor(mask, threshold)

#### Before that, check the dimension of RGB images

In [104]:
from glob import glob
example_file = glob('../../carvana-challenge/input/*.jpg')
example_img = Image.open(example_file[0])    
example_img = np.asarray(example_img.convert("RGB"), dtype=np.float32)

In [110]:
example_img.transpose((2, 0, 1)).shape

(3, 1280, 1918)

#### Step-by-step tensor conversion

In [62]:
mean = 0
std = 1
image = img.astype(np.float32)
image = (image - mean) / std #There is no scaling in this case
image = np.expand_dims(image, axis=0) #(1, 64, 64)
tensor = torch.from_numpy(image)

In [63]:
tensor.size()

torch.Size([1, 64, 64])

In [64]:
mask1 = mask
mask1 = (mask1 > threshold).astype(np.float32)
tensor = torch.from_numpy(mask1).type(torch.FloatTensor)

In [67]:
np.unique(tensor)

array([1.], dtype=float32)

In [68]:
torch.unique(tensor)

tensor([1.])

#### Testing get_mask_representation

In [69]:
"""
Given a mask and an image this method returns
one image representing 3 patches of the same image.
"""

image = img.astype(np.float32)
image = (image - mean) / std #There is no scaling in this case
image = np.expand_dims(image, axis=0) #(1, 64, 64)
image = image.transpose((1, 2, 0))

In [111]:
import cv2
import torch
import numpy as np
import scipy.misc as scipy
from tensorboardX import SummaryWriter

H, W, C = image.shape
results = np.zeros((H, 3 * W, 1), np.uint8)  # shape is (64, 192, 1)
p = np.zeros((H * W, 1), np.uint8)  # shape is (4096, 1)

m = np.zeros((H * W), np.uint8)
l = mask1.reshape(-1)

color = 255
mask2 = np.expand_dims(mask1, axis=2) * np.array(color)
mask2 = mask2.astype(np.uint8)
# masked_img = self._apply_mask_overlay(image, mask)

a = (2 * l + m)
miss = np.where(a == 2)[0]
hit = np.where(a == 3)[0]
fp = np.where(a == 1)[0]

masked_img = cv2.addWeighted(mask, 0.5, image, 0.5, 0.)
p[miss] = np.array([255])
p[hit] = np.array([128])
p[fp] = np.array([0])
p = p.reshape(H, W, 1)
masked_img = np.expand_dims(masked_img, axis = 2)
results[:, 0:W] = image
results[:, W:2 * W] = p
results[:, 2 * W:3 * W] = masked_img

In [113]:
results.shape

(64, 192, 1)

In [79]:
np.unique(a)

array([2.], dtype=float32)